In [1]:
import cv2
import sys
sys.path.append("./TagDetection/")
from apriltag import Apriltag
from matplotlib import pyplot as plt

In [2]:
from readData import *
from showFig import *
from showCloud import *
from warp import *
from transform import Transformer


tf = Transformer()


In [3]:
def plotMarkers(frame, detections, name=None):
    plt.figure().set_size_inches(20, 10)
    plt.imshow(frame)
    plt.gray()
    for detection in detections:
        convex = np.array(detection.points)
        showpoint = np.array(detection.markers)
        plt.plot(showpoint[:, 1], showpoint[:, 0], 'rx')
        plt.plot(convex[:, 0, 0], convex[:, 0, 1], 'b.')
    plt.imshow(frame)
    plt.gray()
    if name:
        plt.savefig(name)
    # plt.show()


def findMarker(num=4):
    ap = Apriltag()
    ap.create_detector(family='tag16h5', debug=False)
    for i in range(num):
        points, frame = getPoints("..\plane\marker%d.npy"%i)
        detections = ap.detect(frame)
        if len(detections) > 0:
            print(len(detections))
            print('%d 识别成功'%i)
            plotMarkers(frame, detections, "marker%d.png"%i)
        else:
            print('%d 识别失败'%i)



In [4]:
ap = Apriltag()
ap.create_detector(family='tag16h5', debug=False)

In [5]:
cloud0, frame = getPoints("..\plane\marker0.npy")
detections = ap.detect(frame)
if len(detections) > 0:
    print(len(detections))
    print('识别成功')
    # plotMarkers(frame, detections)
    points = tf.pix2point(detections[0].points, cloud0)
    tf.tgt = points
else:
    print('识别失败')

read in ..\plane\marker0.npy
x range: -196 ~ 1195
y range: -813 ~ 145
z range: -9869 ~ 364
1
识别成功


In [6]:
cloud1, frame = getPoints("..\plane\marker1.npy")
detections = ap.detect(frame)
if len(detections) > 0:
    print(len(detections))
    print('识别成功')
    # plotMarkers(frame, [detections[0]])
    points = tf.pix2point(detections[0].points, cloud1)
    tf.src = points
else:
    print('识别失败')

read in ..\plane\marker1.npy
x range: -431 ~ 18187
y range: -25187 ~ 676
z range: -129112 ~ 4105
2
识别成功


In [7]:
T = tf.fit()
transCloud = tf.transform(cloud1)

In [8]:
dcloud0 = filterPoints(cloud0, (400, 300, (-100, -30)), skip=20)
dcloud1 = filterPoints(transCloud, (400, 300, (-100, -30)), skip=20)
fig = dynVisual([dcloud0, dcloud1], ["tgt", "src"])
fig.write_html("merge.html")